In [1]:
from dataclasses import dataclass
import numpy as np
from numba import njit, prange, get_thread_id
import numba
import os
import time
import math
from helper_functions import round
from levy_flights import estimate_mean_abs_levy, levy_step, levy_bit_flip,levy_jump_Q
from Swarm_IQ import decode_prod_plan, compute_objective, encode_prod_plan

In [16]:

@dataclass(frozen=True)
class SwarmConfig:
    M: int = 8
    T: int = 8
    setup_costs: np.ndarray = np.array([112,184,144,187,127,147,100,188], dtype=np.float64)
    production_costs: np.ndarray = np.array([1,1,1,1,1,1,1,1], dtype=np.float64)
    inventory_costs: np.ndarray = np.array([5,3,4,6,5,3,6,2], dtype=np.float64)
    production_times: np.ndarray = np.array([1,1,1,1,1,1,1,1], dtype=np.float64)
    setup_times: np.ndarray = np.array([2,5,5,1,8,3,6,2], dtype=np.float64)
    capacities: np.ndarray = np.array([200,200,210,200,160,190,170,170], dtype=np.float64)
    # each row corresponds to one product
    demand: np.ndarray = np.array([[43.0, 29.0, 52.0, 0.0, 0.0, 0.0, 42.0, 0.0],
       [30.0, 0.0, 0.0, 40.0, 20.0, 0.0, 6.0, 27.0],
       [0, 20, 0, 50, 60, 11, 0, 30],
       [33.0, 43.0, 30.0, 0.0, 16.0, 48.0, 37.0, 33.0],
       [0.0, 0.0, 0.0, 41.0, 16.0, 0.0, 55.0, 0.0],
       [0, 0, 21, 13, 7, 0, 22, 0],
       [0.0, 25.0, 43.0, 25.0, 0.0, 52.0, 10.0, 42.0],
       [42.0, 18.0, 40.0, 2.0, 0.0, 71.0, 0.0, 41.0]], dtype=np.float64)


In [17]:
@njit
def iterate(X, Q, VZPK, VQPK,
            pbest_X, pbest_Q, pbest_value,
            gbest_X, gbest_Q,
            inertia, C1, C2, Vzmax, Vqmax,
            demand, production_times, setup_times, capacities,
            production_costs, setup_costs, inventory_costs, 
            r1, r2, r3, r4, rand_vals, stagn_count, levy_Sx, levy_Sq, levy_alpha,levy_q_step,stagn_thres,  particle_idx):
    M, T = X.shape
    stagnation = stagn_count[particle_idx]
    
    # perform a levy flight after particle stagnated for a certain time
    if stagnation > stagn_thres:
        
        X = levy_bit_flip(pbest_X, levy_Sx, levy_alpha)
        Q = levy_jump_Q(pbest_Q, levy_Sq, levy_q_step, levy_alpha)
        

        stagnation = 0.0
        
    # perform regular update of particle
    else:
        # update VZPK
        for i in range(M):
            for j in range(T):
                VZPK[i, j] = inertia * VZPK[i, j] \
                             + C1 * r1 * (pbest_X[i, j] - X[i, j]) \
                             + C2 * r2 * (gbest_X[i, j] - X[i, j])
                if VZPK[i, j] > Vzmax:
                    VZPK[i, j] = Vzmax
                elif VZPK[i, j] < -Vzmax:
                    VZPK[i, j] = -Vzmax
    
        # update VQPK
        for i in range(M):
            for j in range(T):
                VQPK[i, j] = inertia * VQPK[i, j] \
                             + C1 * r3 * (pbest_Q[i, j] - Q[i, j]) \
                             + C2 * r4 * (gbest_Q[i, j] - Q[i, j])
                if VQPK[i, j] > Vqmax:
                    VQPK[i, j] = Vqmax
                elif VQPK[i, j] < -Vqmax:
                    VQPK[i, j] = -Vqmax
    
        # update X
        for i in range(M):
            for j in range(T):
                if 1.0 / (1.0 + np.exp(-VZPK[i, j])) > rand_vals[i, j]:
                    X[i, j] = 1.0
                else:
                    X[i, j] = 0.0
    
        for i in range(M):
            for j in range(T):
                Q[i, j] = Q[i, j] + VQPK[i, j]


        

    # decode and evaluate
    prod_quant = decode_prod_plan(X.T, Q.T, demand)
    X_float = X.astype(np.float64)
    fitness = compute_objective(
        prod_quant, X_float,
        setup_costs, production_costs,
        production_times, setup_times,
        capacities, inventory_costs, demand
    )

    # unpack scalar fitness and pbest
    fv0 = fitness[0]
    fv1 = fitness[1]
    bv0 = pbest_value[0, 0]
    bv1 = pbest_value[0, 1]

    # update personal best
    if (fv0 < bv0) or (fv0 == bv0 and fv1 < bv1):
        for ii in range(M):
            for jj in range(T):
                pbest_X[ii, jj] = X[ii, jj]
                pbest_Q[ii, jj] = Q[ii, jj]
        pbest_value[0, 0] = fv0
        pbest_value[0, 1] = fv1
    else:
        stagnation += 1.0

    return X, Q, VZPK, VQPK, pbest_X, pbest_Q, pbest_value, stagnation


<!-- Pseudo-code for update_swarm -->
<ol>
  <li>Let <code>num_threads</code> ← number of parallel threads</li>
  <li>Initialize arrays:
    <ul>
      <li><code>thread_best_values[num_threads]</code> ← ∞</li>
      <li><code>thread_best_indices[num_threads]</code> ← 0</li>
    </ul>
  </li>
  <li>Let <code>N</code> ← number of particles</li>
  <li>Parallel loop over <code>i</code> from 0 to <code>N–1</code>:
    <ol type="a">
      <li>Call <code>iterate</code> on particle <code>i</code>, passing its state and precomputed random values 
        → returns updated state and personal best <code>pbest_value</code>
      </li>
      <li>Write back updated state into global arrays</li>
      <li>Let <code>t</code> ← thread ID</li>
      <li>If <code>pbest_value &lt; thread_best_values[t]</code>:
        <ul>
          <li><code>thread_best_values[t]</code> ← <code>pbest_value</code></li>
          <li><code>thread_best_indices[t]</code> ← <code>i</code></li>
        </ul>
      </li>
    </ol>
  </li>
  <li>Return (<code>thread_best_values</code>, <code>thread_best_indices</code>)</li>
</ol>


In [18]:
@njit(parallel=True)
def update_swarm(Xs, Qs, VXs, VQs, pbest_X, pbest_Q, pbest_Val,
                 gbest_X, gbest_Q, inertia, C1, C2, Vzmax, Vqmax,
                 demand, production_times, setup_times, capacities,
                 production_costs, setup_costs, inventory_costs,
                 r1_arr, r2_arr, r3_arr, r4_arr, rand_vals, stagn_count, levy_Sx, levy_Sq, levy_alpha, levy_q_step,stagn_thres):

    nthreads = numba.get_num_threads()
    local_best = np.full((nthreads, 1, 2), np.inf)
    local_idx  = np.zeros(nthreads, np.int64)
    N = Xs.shape[0]

    for i in prange(N):
        Xi, Qi, VXi, VQi, pbest_Xi, pbest_Qi, pbest_Values, stagn_val_i = iterate(
            Xs[i], Qs[i], VXs[i], VQs[i],
            pbest_X[i], pbest_Q[i], pbest_Val[i],
            gbest_X, gbest_Q,
            inertia, C1, C2, Vzmax, Vqmax,
            demand, production_times, setup_times, capacities,
            production_costs, setup_costs, inventory_costs, 
            r1_arr[i], r2_arr[i], r3_arr[i], r4_arr[i],
            rand_vals[i], stagn_count, levy_Sx, levy_Sq, levy_alpha, levy_q_step, stagn_thres,  particle_idx = i
        )
        # update swarm in place 
        Xs[i], Qs[i], VXs[i], VQs[i] = Xi, Qi, VXi, VQi
        pbest_X[i], pbest_Q[i], pbest_Val[i] = pbest_Xi, pbest_Qi, pbest_Values

        # write back the updated stagnation count
        stagn_count[i] = stagn_val_i


        t = numba.get_thread_id()
        pv0 = pbest_Values[0, 0]
        pv1 = pbest_Values[0, 1]
        lb0 = local_best[t, 0, 0]
        lb1 = local_best[t, 0, 1]
        if (pv0 < lb0) or (pv0 == lb0 and pv1 < lb1):
            local_best[t, 0, 0] = pv0
            local_best[t, 0, 1] = pv1
            local_idx[t]       = i

    return local_best, local_idx


In [19]:
class Swarm:
    def __init__(self, n_particles, C1=1.5, C2=1.5, Vzmax=4.0, Vqmax=0.1, inertia = 1.0, stagn_thres = 49,
                 levy_alpha = 1.5, levy_c_frac = 0.1, levy_samples=100000, levy_q_max_step=1.0, target_avg_Q_step = 0.1):
        # keep the frozen config around for the other parameters
        self.cfg = SwarmConfig()
        N, M, T       = n_particles, self.cfg.M, self.cfg.T

        self.levy_samples = levy_samples
        self.levy_alpha   = levy_alpha
        self.levy_c_frac  = levy_c_frac
        self.levy_q_max_step = levy_q_max_step
        self.stagn_thres = stagn_thres

        # precompute mean |L| and scales
        mean_abs_L = estimate_mean_abs_levy(self.levy_alpha, self.levy_samples)
        #n_bits     = N * T * M
        #n_bits     =  T * M
        #self.levy_Sx = (self.levy_c_frac * n_bits) / mean_abs_L
        self.levy_Sx = (self.levy_c_frac ) / mean_abs_L

        #D = T * M
        #c_Q = self.levy_c_frac * D
        #self.levy_Sq = c_Q / mean_abs_L
        self.levy_Sq = target_avg_Q_step / mean_abs_L

        self.inertia = inertia
        self.C1, self.C2, self.Vzmax, self.Vqmax = C1, C2, Vzmax, Vqmax

        # initialize all particles using 3D arrays
        self.Xs   = np.random.randint(0, 2, size=(N, T, M)).astype(np.float64)
        #self.Qs = np.random.uniform(-1.0, 1.0, size=(N, T, M))
        self.Qs = np.random.rand(N, T, M)
        self.VZs  = (np.random.rand(N, T, M)*2 - 1) * Vzmax
        self.VQs  = (np.random.rand(N, T, M)*2 - 1) * Vqmax

        self.pbest_X   = self.Xs.copy()
        self.pbest_Q   = self.Qs.copy()
        self.pbest_Val = np.full((N,  1, 2), np.inf)

        self.gbest_X   = np.zeros((T, M), dtype=np.float64)
        self.gbest_Q   = np.zeros((T, M), dtype=np.float64)
        self.gbest_Val = np.full((1,2), np.inf)
        self.stagn_count = np.zeros(N, dtype=np.int16)


        # one cold-start update to fill pbest and gbest
        self._reduce_global(update_swarm(
            self.Xs, self.Qs, self.VZs, self.VQs,
            self.pbest_X, self.pbest_Q, self.pbest_Val,
            self.gbest_X, self.gbest_Q,
            self.inertia, C1, C2, Vzmax, Vqmax,
            self.cfg.demand,
            self.cfg.production_times,
            self.cfg.setup_times,
            self.cfg.capacities,
            self.cfg.production_costs,
            self.cfg.setup_costs,
            self.cfg.inventory_costs,
            # no randomness needed for cold start: pass zeros
            np.zeros(N), np.zeros(N), np.zeros(N), np.zeros(N),
            np.zeros((N, T, M)), self.stagn_count, self.levy_Sx, self.levy_Sq, self.levy_alpha, 
            self.levy_q_max_step,  self.stagn_thres
        ))

    def _reduce_global(self, thread_results):
        # reduces all the thread-local minima into a single true global minimum
        values, indices = thread_results
        for t in range(values.shape[0]):
            i = indices[t]
            v0 = values[t, 0, 0]
            v1 = values[t, 0, 1]
            b0 = self.gbest_Val[0, 0]
            b1 = self.gbest_Val[0, 1]
            if (v0 < b0) or (v0 == b0 and v1 < b1):
                self.gbest_Val[0, 0] = v0
                self.gbest_Val[0, 1] = v1
                self.gbest_X[:] = self.pbest_X[i]
                self.gbest_Q[:] = self.pbest_Q[i]

    def optimize(self, n_iter):
        
        N, T, M = self.Xs.shape
    
        for t in range(n_iter):
            if (t==1000):
                self.Vqmax = 0.1
 
            # pre-generate random numbers
            r1, r2, r3, r4 = (np.random.rand(N) for _ in range(4))
            rand_vals     = np.random.rand(N, T, M)

            # perform iteration of entire swarm
            thread_results = update_swarm(
                self.Xs, self.Qs, self.VZs, self.VQs,
                self.pbest_X, self.pbest_Q, self.pbest_Val,
                self.gbest_X, self.gbest_Q,
                self.inertia, self.C1, self.C2, self.Vzmax, self.Vqmax,
                self.cfg.demand, self.cfg.production_times,
                self.cfg.setup_times, self.cfg.capacities,
                self.cfg.production_costs, self.cfg.setup_costs, self.cfg.inventory_costs,
                r1, r2, r3, r4, rand_vals, self.stagn_count, self.levy_Sx, self.levy_Sq,
                self.levy_alpha, self.levy_q_max_step,  self.stagn_thres
            )
            # update global best solution
            self._reduce_global(thread_results)
    
        return decode_prod_plan(self.gbest_X.T, self.gbest_Q.T,self.cfg.demand) ,self.gbest_Val
        
    def update_params(self,levy_c_frac,levy_q_step,Vzmax,Vqmax, inertia ):
        self.levy_c_frac  = levy_c_frac
        self.levy_q_step = levy_q_step
        self.Vzmax = Vzmax
        self.Vqmax = Vqmax
        self.inertia = inertia 


    def get_params(self):
        print('C1', self.C1)
        print('C2', self.C2)
        print('inertia', self.inertia)
        print('Vzmax', self.Vzmax)
        print('Vqmax', self.Vqmax)

        

In [33]:
demand = np.array([[10., 20., 20.,  0., 35., 65.],
       [20.,  0.,  3., 18., 13., 65.],
       [ 2., 10., 12., 30.,  0.,  0.]])


#  pin a stable threading layer before Numba is loaded:
os.environ["NUMBA_THREADING_LAYER"] = "workqueue"
results = []
bad_results = []
for i in range(20):

    np.random.seed(i) 
    # instantiate swarm
    
    sw = Swarm(n_particles=100, C1=2.0, C2=2.0, Vzmax=4.0, Vqmax=0.2, inertia = 1.0, levy_c_frac = 0.1, levy_q_max_step=20.0, target_avg_Q_step = 0.0, stagn_thres = 4400)
    # timing
    
    start = time.perf_counter()
    #plan, best_val = sw.optimize(n_iter=0)
    end = time.perf_counter()
    
    plan, best_val = sw.optimize(n_iter=2000)
    #X, Q = encode_prod_plan(plan.T,demand)
    #new_plan = decode_prod_plan(X.T,Q.T,demand)
    
    #if np.any(abs(plan - new_plan)>1):
        #print("true plan\n", plan)
        #print("encoded plan\n", new_plan)
        #print('-------------------------')
     
    #print(f"Elapsed time: {end - start:.4f} seconds")
    if best_val[0][0]==0.0:
        results.append(best_val[0][1])
   
    else:
        #print(i)
        bad_results.append((plan,best_val))

print(results)

[9270.0, 8411.0, 7986.0, 8501.0, 8624.0, 7867.0, 7240.0, 7679.0, 8693.0, 8275.0, 9792.0, 8521.0, 8392.0, 7563.0, 7326.0, 9226.0, 7118.0, 8572.0, 7410.0]


In [34]:
print(np.percentile(results,0))
print( np.percentile(results,5))
print( np.percentile(results,25))
print(np.percentile(results,50))
print( np.percentile(results,75))
print( np.percentile(results,90))
print(np.percentile(results,95))
print(np.percentile(results,100))

7118.0
7227.8
7621.0
8392.0
8598.0
9234.8
9322.199999999999
9792.0


In [35]:
len(bad_results)

1